In [1]:
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import KFold, cross_validate
from surprise import BaselineOnly, NormalPredictor, KNNBasic, SVD, SVDpp, SlopeOne

In [2]:
# 设置Reader类
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
# 从文件中读取评分数据
data = Dataset.load_from_file('../ratings.csv', reader = reader)

In [3]:
# 交叉验证        
def cross_validation(algo, data, measures=['RMSE', 'MAE'], n_splits=5, verbose=True):
    # 打印信息更全面
    cross_validate(algo, data, measures=measures, cv=n_splits, verbose=verbose)

##### BaselineOnly
###### 根据所有用户的评分均值mu，用户u整体的评分偏差bu和反映商品i受欢迎程度的bi，预测出用户u对商品i的评分bui=mu-bu-bi

In [4]:
# model definition
# baseline estimate configuration
bsl_options = {'method': 'als',                   # 利用ALS进行优化
               'n_epochs': 5,                     # 迭代次数
               'reg_u': 12,                       # user矩阵的惩罚系数
               'reg_i': 5,                        # item矩阵的惩罚系数
               }
bsl_only = BaselineOnly(bsl_options=bsl_options)

cross_validation(bsl_only, data, n_splits=5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8640  0.8630  0.8599  0.8617  0.8622  0.8621  0.0014  
MAE (testset)     0.6637  0.6643  0.6617  0.6635  0.6645  0.6635  0.0010  
Fit time          1.59    1.93    1.96    2.07    1.93    1.90    0.16    
Test time         1.50    1.50    1.53    1.68    1.50    1.54    0.07    


##### NormalPredictor
###### 根据训练集所确定的正态分布随机预测一个评分

In [5]:
norm_predictor = NormalPredictor()
cross_validation(norm_predictor, data)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4297  1.4337  1.4326  1.4287  1.4317  1.4313  0.0018  
MAE (testset)     1.1417  1.1445  1.1425  1.1408  1.1427  1.1424  0.0013  
Fit time          1.11    1.61    1.45    1.53    1.46    1.43    0.17    
Test time         2.07    1.99    2.04    2.06    2.12    2.06    0.04    


##### Slope One
###### 根据已有的评分寻找规律,应用上述规律对待预测商品进行评分

In [6]:
slope_one = SlopeOne()
cross_validation(slope_one, data, n_splits=2)

Evaluating RMSE, MAE of algorithm SlopeOne on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    0.8730  0.8736  0.8733  0.0003  
MAE (testset)     0.6694  0.6706  0.6700  0.0006  
Fit time          11.77   11.88   11.82   0.05    
Test time         112.85  105.21  109.03  3.82    


##### funkSVD
###### SVD矩阵分解虽然形式上是分解成三个矩阵，但中间的特征对角阵可以融合到另外两个矩阵中，即简化成优化p,q两个矩阵。实现方式为surprise.SVD的biased=False模式。

In [7]:
%%time
funksvd = SVD(n_factors=50, n_epochs=20, biased=False, lr_all=0.005, reg_all=0.02)
cross_validation(funksvd, data)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8482  0.8520  0.8525  0.8520  0.8504  0.8510  0.0016  
MAE (testset)     0.6492  0.6518  0.6510  0.6509  0.6505  0.6507  0.0009  
Fit time          32.82   31.21   31.78   31.13   30.44   31.48   0.80    
Test time         2.48    2.14    2.18    2.49    2.18    2.29    0.16    
Wall time: 2min 57s


##### BiasSVD
###### 类似baselineonly引入用户对商品评分的均值mu，用户u的评分偏差bu和商品i的(受欢迎程度、质量等)偏差bi，优化变量的数目从funkSVD的2个增加到4个(p,q,bu,bi)

In [8]:
%%time
biassvd = SVD(n_factors=50, n_epochs=20, biased=True, lr_all=0.005, reg_all=0.02)
cross_validation(biassvd, data)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8312  0.8318  0.8320  0.8313  0.8333  0.8319  0.0008  
MAE (testset)     0.6362  0.6362  0.6366  0.6361  0.6375  0.6365  0.0005  
Fit time          31.50   31.74   29.94   30.06   30.63   30.77   0.73    
Test time         2.56    2.95    2.16    2.59    2.59    2.57    0.25    
Wall time: 2min 55s


##### SVD++
###### 除了用户和商品的偏差外，现实数据中还存在着大量隐式反馈的信息，这些信息提供了用户爱好的额外提示。因此SVD++在BiasSVD的基础上添加了隐式反馈因子，优化变量较之前增加了一个，训练时长也相应增加。

In [9]:
%%time
svdpp = SVDpp(n_factors=50, n_epochs=20,  lr_all=0.005, reg_all=0.02)
cross_validation(svdpp, data, n_splits=2)

Evaluating RMSE, MAE of algorithm SVDpp on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    0.8451  0.8469  0.8460  0.0009  
MAE (testset)     0.6466  0.6476  0.6471  0.0005  
Fit time          2784.77 2742.40 2763.59 21.18   
Test time         120.12  117.75  118.93  1.18    
Wall time: 1h 36min 9s


##### 基于邻域的协同过滤
###### 本质：计算相似度！！！

##### ItemCF

In [10]:
%%time
# 'pearson_baseline':shrunk Pearson correlation coefficient,
# the shrinkage parameter helps to avoid overfitting when only few rating are available.
sim_options = {'name': 'pearson_baseline',             
              'user_based': False}
itemcf = KNNBasic(sim_options = sim_options)                     # 默认用的als?
cross_validation(itemcf, data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8787  0.8765  0.8751  0.8789  0.8767  0.8772  0.0014  
MAE (testset)     0.6668  0.6655  0.6644  0.6673  0.6656  0.6659  0.0011  
Fit time          65.62   68.22   69.76   68.27   65.40   67.46   1.69    
Test time         66.12   65.17   64.92   65.81   65.96 

##### UserCF

In [13]:
%%time
sim_options = {'name': 'pearson_baseline',
              'user_base': True}
usercf = KNNBasic(sim_options=sim_options)
cross_validation(usercf, data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9049  0.9009  0.9033  0.9033  0.9029  0.9031  0.0013  
MAE (testset)     0.7000  0.6975  0.6991  0.6991  0.6990  0.6990  0.0008  
Fit time          86.31   86.59   86.40   86.04   87.25   86.52   0.41    
Test time         105.49  98.68   97.75   99.57   108.30